In [2]:
import torch
from peft import PeftModel, PeftConfig
import transformers
import os, time
import tempfile
from torch.utils.data import DataLoader
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
    Seq2SeqTrainer,
    AutoTokenizer,
)

BASE_MODEL = "openai/whisper-small"
LORA_WEIGHTS = "sin2piusc/whisper-small-CV16-GF-AS-jp"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = WhisperForConditionalGeneration.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
)

model = PeftModel.from_pretrained(
    model,
    LORA_WEIGHTS,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model")

model.save_pretrained("/whisper-small-CV16-GF-AS-MERGED")
#model.push_to_hub("./whisper-small-CV16-GF-AS-MERGED")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
